# Pgmpy를 사용한 베이즈 정리 적용
- pgmpy 패키지는 베이즈 정리에 적용하는 BayesianModel 클래스 제공
- 베이즈 정리를 적용하려면 조건부확률을 구현하는 TabularCPD 클래스를 사용하여 사전확률 및 가능도를 구현해야 함

```python
TabularCPD(variable, variable_card, value, evidence=None, evidence_card=None)
```
- `variable` : 확률변수의 이름 문자열
- `variable_card` : 확률변수가 가질 수 있는 경우의 수
- `value` : 조건부확률 배열, 하나의 열(column)이 동일 조건을 뜻하므로 하나의 열의 확률 합은 1이어야 함
- `evidence` : 조건이 되는 확률변수의 이름 문자열 리스트
- `evidence_card` : 조건이 되는 확률변수가 가질 수 있는 경우의 수 리스트

`TabularCPD` 클래스는 원래는 조건부확률을 구현하기 위한 것이지만, `evidence=None`, `evidence_card=None`으로 인수를 주면 일반적인 확률도 구현할 수 있다.

In [6]:
from pgmpy.factors.discrete import TabularCPD
import numpy as np

In [4]:
# 병에 걸렸을 사전확률 P(D) = P(X=1)
# 병에 걸리지 않았을 사전확률 P(D^C) = P(X=0)

cpd_X = TabularCPD('X', 2, [[1-0.002], [0.002]])
print(cpd_X)

+------+-------+
| X(0) | 0.998 |
+------+-------+
| X(1) | 0.002 |
+------+-------+


In [8]:
# 양성 반응 나올 확률 P(S) = P(Y=1)
# 음성 반응 나올 확률 P(S^C) = P(Y=0)
# 확률변수 Y의 확률을 베이즈 모형에 넣을 때는 
# TabularCPD 클래스를 사용한 조건부확률 P(Y|X) 형태로 넣어야 하므로
# 다음처럼 조건부확률 P(Y|X)를 구현한다.

cpd_Y_on_X = TabularCPD('Y', 2, np.array([[0.95, 0.01], [0.05, 0.99]]),
                  evidence=['X'], evidence_card=[2])
print(cpd_Y_on_X)

+------+------+------+
| X    | X(0) | X(1) |
+------+------+------+
| Y(0) | 0.95 | 0.01 |
+------+------+------+
| Y(1) | 0.05 | 0.99 |
+------+------+------+


### `BayesianModel` 클래스 객체
- 확률변수들이 어떻게 결합되어 있는지 나타내는 확률모형
- `BayesianModel(variables)`

In [11]:
from pgmpy.models import BayesianModel

model = BayesianModel([('X', 'Y')])
model.add_cpds(cpd_X, cpd_Y_on_X)
model.check_model()

True

- `BayesianModel` 클래스는 변수 제거법(Variable Elimination)을 사용한 추정을 제공한다.
- `VariableEliminiation` 클래스로 추정(inference) 객체를 만들고 `query()` 메서드를 사용하면 사후확률을 계산한다.

```python
query(variables, evidences)
```
- `variables` : 사후확률을 계산할 확률변수의 이름 리스트
- `evidences` : 조건이 되는 확률변수의 값을 나타내는 딕셔너리

여기에서는 pgmpy 패키지를 이용하여 베이즈 정리를 적용할 수 있다는 것만 알면 된다. 자세한 내용은 추후 확률적 그래프 모형(Probabilistic Graphical Model)에서 다룬다.

In [13]:
from pgmpy.inference import VariableElimination
inference = VariableElimination(model)
posterior = inference.query(['X'], evidence={'Y':1}, joint=False, show_progress=True)
print(posterior['X'])

Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]

+------+----------+
| X    |   phi(X) |
+======+==========+
| X(0) |   0.9618 |
+------+----------+
| X(1) |   0.0382 |
+------+----------+
